In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os, random, time, glob
import numpy as np
import tensorflow as tf

print("TensorFlow:", tf.__version__)

SEED = 1337
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

try:
    tf.keras.mixed_precision.set_global_policy("mixed_float16")
    print("Mixed precision: ON")
except Exception as e:
    print("Mixed precision not available:", e)

DATA_DIR  = "/kaggle/input/gan-getting-started"
PHOTO_JPG = os.path.join(DATA_DIR, "photo_jpg")
MONET_JPG = os.path.join(DATA_DIR, "monet_jpg")

IMG_SIZE   = 256
IMG_SHAPE  = (IMG_SIZE, IMG_SIZE, 3)
BATCH_SIZE = 4  


EPOCHS       = 50
LR           = 2e-4
LAMBDA_CYCLE = 10.0
LAMBDA_ID    = 0.5

AUTO = tf.data.AUTOTUNE


In [ ]:
def decode_jpg(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)          # RGB
    img = tf.image.convert_image_dtype(img, tf.float32)  # [0,1]
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE], method="area")
    return img

@tf.function
def augment(img):

    j = IMG_SIZE + 30
    img = tf.image.resize(img, [j, j], method="bilinear")
    img = tf.image.random_crop(img, [IMG_SIZE, IMG_SIZE, 3])
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.05)
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)
    return img

@tf.function
def to_m11(img): 
    return img * 2.0 - 1.0

def load_jpg_dataset(folder, training=True):
    files = tf.io.gfile.glob(os.path.join(folder, "*.jpg"))
    files.sort()
    ds = tf.data.Dataset.from_tensor_slices(files)
    if training:
        ds = ds.shuffle(len(files), seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(decode_jpg, num_parallel_calls=AUTO)
    if training:
        ds = ds.map(augment, num_parallel_calls=AUTO)
    ds = ds.map(to_m11, num_parallel_calls=AUTO)
    ds = ds.batch(BATCH_SIZE, drop_remainder=training)  
    ds = ds.prefetch(AUTO)
    return ds


photo_ds = load_jpg_dataset(PHOTO_JPG, training=True)
monet_ds = load_jpg_dataset(MONET_JPG, training=True)

print("photo_ds batches:", tf.data.experimental.cardinality(photo_ds).numpy())
print("monet_ds batches:", tf.data.experimental.cardinality(monet_ds).numpy())


In [ ]:

from tensorflow.keras import layers as KL, Model

class InstanceNorm(KL.Layer):
    def __init__(self, eps=1e-5):
        super().__init__(); self.eps = eps
    def build(self, s):
        ch = s[-1]
        self.gamma = self.add_weight(shape=(ch,), initializer="ones",  trainable=True)
        self.beta  = self.add_weight(shape=(ch,), initializer="zeros", trainable=True)
    def call(self, x):
        m, v = tf.nn.moments(x, axes=[1,2], keepdims=True)
        return self.gamma * (x - m) / tf.sqrt(v + self.eps) + self.beta

def conv7(x, f):
    x = KL.Conv2D(f, 7, padding="same", use_bias=False)(x)
    x = InstanceNorm()(x)
    return KL.Activation("relu")(x)

def down(x, f):
    x = KL.Conv2D(f, 3, strides=2, padding="same", use_bias=False)(x)
    x = InstanceNorm()(x)
    return KL.Activation("relu")(x)

def resblock(x, f):
    y = KL.Conv2D(f, 3, padding="same", use_bias=False)(x)
    y = InstanceNorm()(y); y = KL.Activation("relu")(y)
    y = KL.Conv2D(f, 3, padding="same", use_bias=False)(y)
    y = InstanceNorm()(y)
    return KL.Add()([x, y])

def up(x, f):
    x = KL.Conv2DTranspose(f, 3, strides=2, padding="same", use_bias=False)(x)
    x = InstanceNorm()(x)
    return KL.Activation("relu")(x)

def build_generator(img_shape=IMG_SHAPE, n_res=9):
    inp = KL.Input(img_shape)
    x = conv7(inp, 64)
    x = down(x, 128); x = down(x, 256)
    for _ in range(n_res): x = resblock(x, 256)
    x = up(x, 128); x = up(x, 64)
    x = KL.Conv2D(3, 7, padding="same")(x)
    x = KL.Activation("tanh")(x)                              
    out = KL.Lambda(lambda t: tf.cast(t, tf.float32))(x)      
    return Model(inp, out, name="Generator")

def build_discriminator(img_shape=IMG_SHAPE):
    def blk(x, f, s, norm=True):
        x = KL.Conv2D(f, 4, strides=s, padding="same", use_bias=not norm)(x)
        if norm: x = InstanceNorm()(x)
        return KL.LeakyReLU(0.2)(x)
    inp = KL.Input(img_shape)
    x = blk(inp,  64, 2, norm=False)
    x = blk(x,   128, 2)
    x = blk(x,   256, 2)
    x = blk(x,   512, 1)
    out = KL.Conv2D(1, 4, padding="same")(x)  
    return Model(inp, out, name="Discriminator")

# 2x Generatory i 2x Dyskryminatory
G_p2m = build_generator()    
G_m2p = build_generator()     
D_m   = build_discriminator() 
D_p   = build_discriminator() 


In [ ]:
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()

def gen_gan_loss(d_fake):
  
    return mse(tf.ones_like(d_fake), d_fake)

def disc_loss(d_real, d_fake):
   
    loss_real = mse(tf.ones_like(d_real), d_real)
    loss_fake = mse(tf.zeros_like(d_fake), d_fake)
    return 0.5 * (loss_real + loss_fake)

opt_G  = tf.keras.optimizers.Adam(LR, beta_1=0.5, beta_2=0.999)
opt_Dm = tf.keras.optimizers.Adam(LR, beta_1=0.5, beta_2=0.999)
opt_Dp = tf.keras.optimizers.Adam(LR, beta_1=0.5, beta_2=0.999)

CKPT_DIR = os.path.join("/kaggle/working", "ckpts")
ckpt = tf.train.Checkpoint(
    G_p2m=G_p2m, G_m2p=G_m2p, D_m=D_m, D_p=D_p,
    opt_G=opt_G, opt_Dm=opt_Dm, opt_Dp=opt_Dp
)
ckpt_manager = tf.train.CheckpointManager(ckpt, directory=CKPT_DIR, max_to_keep=3)

latest = ckpt_manager.latest_checkpoint
if latest:
    print(f"Przywracam checkpoint: {latest}")
    ckpt.restore(latest)
else:
    print("Brak istniejących checkpointów — start od zera.")


In [ ]:
# %% ETAP 4 — train_step 
@tf.function
def train_step(batch_photo, batch_monet):
    with tf.GradientTape(persistent=True) as tape:
     
        fake_monet = G_p2m(batch_photo, training=True)
        fake_photo = G_m2p(batch_monet, training=True)

        
        rec_photo = G_m2p(fake_monet, training=True)  
        rec_monet = G_p2m(fake_photo, training=True)  

       
        id_monet = G_p2m(batch_monet, training=True)
        id_photo = G_m2p(batch_photo, training=True)

        
        d_m_real = D_m(batch_monet, training=True)
        d_m_fake = D_m(fake_monet, training=True)
        d_p_real = D_p(batch_photo, training=True)
        d_p_fake = D_p(fake_photo, training=True)

        
        g_adv_p2m = gen_gan_loss(d_m_fake)
        g_adv_m2p = gen_gan_loss(d_p_fake)
        cycle     = (mae(batch_photo, rec_photo) + mae(batch_monet, rec_monet)) * LAMBDA_CYCLE
        ident     = (mae(batch_monet, id_monet) + mae(batch_photo, id_photo)) * (LAMBDA_CYCLE * LAMBDA_ID)
        g_total   = g_adv_p2m + g_adv_m2p + cycle + ident

       
        d_m = disc_loss(d_m_real, d_m_fake)
        d_p = disc_loss(d_p_real, d_p_fake)

    vars_G = G_p2m.trainable_variables + G_m2p.trainable_variables
    opt_G.apply_gradients(zip(tape.gradient(g_total, vars_G), vars_G))
    opt_Dm.apply_gradients(zip(tape.gradient(d_m, D_m.trainable_variables), D_m.trainable_variables))
    opt_Dp.apply_gradients(zip(tape.gradient(d_p, D_p.trainable_variables), D_p.trainable_variables))

    return {"g_total": g_total, "cycle": cycle, "id": ident, "d_m": d_m, "d_p": d_p}


In [ ]:
# %% ETAP 5 — trening: 
LOG_EVERY = 200  

if 'photo_ds' not in globals() or 'monet_ds' not in globals():
    raise RuntimeError("Brak datasetów: uruchom wcześniej ETAP 1.")
if 'train_step' not in globals():
    raise RuntimeError("Brak train_step: uruchom wcześniej ETAP 4.")

global_step = 0
for epoch in range(1, EPOCHS + 1):
    t0 = time.time()


    zipped = tf.data.Dataset.zip((photo_ds, monet_ds)).prefetch(AUTO)

    for step, (photos, monets) in enumerate(zipped, start=1):
        logs = train_step(photos, monets)
        global_step += 1

        if global_step % LOG_EVERY == 0:
            print(
                f"[ep {epoch:02d} | step {global_step:05d}] "
                f"G_total:{float(logs['g_total']):.3f}  "
                f"cycle:{float(logs['cycle']):.3f}  id:{float(logs['id']):.3f}  "
                f"Dm:{float(logs['d_m']):.3f}  Dp:{float(logs['d_p']):.3f}"
            )

    ckpt_manager.save()
    print(f"Epoch {epoch} done in {time.time()-t0:.1f}s  |  checkpoint: {ckpt_manager.latest_checkpoint}")


In [ ]:
import os, tensorflow as tf
from tensorflow.keras import layers as KL, Model

IMG_SIZE  = 256
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
LR = 2e-4 

class InstanceNorm(KL.Layer):
    def __init__(self, eps=1e-5): super().__init__(); self.eps = eps
    def build(self, s):
        ch = s[-1]
        self.gamma = self.add_weight(shape=(ch,), initializer="ones",  trainable=True)
        self.beta  = self.add_weight(shape=(ch,), initializer="zeros", trainable=True)
    def call(self, x):
        m, v = tf.nn.moments(x, axes=[1,2], keepdims=True)
        return self.gamma * (x - m) / tf.sqrt(v + self.eps) + self.beta

def conv7(x, f):
    x = KL.Conv2D(f, 7, padding="same", use_bias=False)(x)
    x = InstanceNorm()(x)
    return KL.Activation("relu")(x)

def down(x, f):
    x = KL.Conv2D(f, 3, strides=2, padding="same", use_bias=False)(x)
    x = InstanceNorm()(x)
    return KL.Activation("relu")(x)

def resblock(x, f):
    y = KL.Conv2D(f, 3, padding="same", use_bias=False)(x)
    y = InstanceNorm()(y); y = KL.Activation("relu")(y)
    y = KL.Conv2D(f, 3, padding="same", use_bias=False)(y)
    y = InstanceNorm()(y)
    return KL.Add()([x, y])

def up(x, f):
    x = KL.Conv2DTranspose(f, 3, strides=2, padding="same", use_bias=False)(x)
    x = InstanceNorm()(x)
    return KL.Activation("relu")(x)

def build_generator(img_shape=IMG_SHAPE, n_res=9):
    inp = KL.Input(img_shape)
    x = conv7(inp, 64)
    x = down(x, 128); x = down(x, 256)
    for _ in range(n_res): x = resblock(x, 256)
    x = up(x, 128); x = up(x, 64)
    x = KL.Conv2D(3, 7, padding="same")(x)
    x = KL.Activation("tanh")(x)                   
    out = KL.Lambda(lambda t: tf.cast(t, tf.float32))(x)
    return Model(inp, out, name="Generator")

def build_discriminator(img_shape=IMG_SHAPE):
    def blk(x, f, s, norm=True):
        x = KL.Conv2D(f, 4, strides=s, padding="same", use_bias=not norm)(x)
        if norm: x = InstanceNorm()(x)
        return KL.LeakyReLU(0.2)(x)
    inp = KL.Input(img_shape)
    x = blk(inp,  64, 2, norm=False)
    x = blk(x,   128, 2)
    x = blk(x,   256, 2)
    x = blk(x,   512, 1)
    out = KL.Conv2D(1, 4, padding="same")(x)
    return Model(inp, out, name="Discriminator")

G_p2m = build_generator()    
G_m2p = build_generator()     
D_m   = build_discriminator()
D_p   = build_discriminator()

opt_G  = tf.keras.optimizers.Adam(LR, beta_1=0.5, beta_2=0.999)
opt_Dm = tf.keras.optimizers.Adam(LR, beta_1=0.5, beta_2=0.999)
opt_Dp = tf.keras.optimizers.Adam(LR, beta_1=0.5, beta_2=0.999)

CKPT_DIR = "/kaggle/working/ckpts"
ckpt = tf.train.Checkpoint(G_p2m=G_p2m, G_m2p=G_m2p, D_m=D_m, D_p=D_p,
                           opt_G=opt_G, opt_Dm=opt_Dm, opt_Dp=opt_Dp)
ckpt_manager = tf.train.CheckpointManager(ckpt, directory=CKPT_DIR, max_to_keep=3)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
    print("Przywrócono:", ckpt_manager.latest_checkpoint)
else:
    raise RuntimeError("Nie znaleziono checkpointów w /kaggle/working/ckpts — uruchom ponownie trening (ETAP 5).")


In [ ]:
import os, zipfile
import tensorflow as tf

PHOTO_JPG = "/kaggle/input/gan-getting-started/photo_jpg"

def load_jpg_dataset_infer(folder, batch_size=4):
    files = tf.io.gfile.glob(os.path.join(folder, "*.jpg"))
    files.sort()
    ds = tf.data.Dataset.from_tensor_slices(files)
    
    def decode_norm(p):
        img = tf.io.read_file(p)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [256, 256])
        img = tf.image.convert_image_dtype(img, tf.float32)  # [0,1]
        return img * 2.0 - 1.0                                # [-1,1]
    
    ds = ds.map(decode_norm, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size, drop_remainder=False).prefetch(tf.data.AUTOTUNE)
    return ds

photo_ds_infer = load_jpg_dataset_infer(PHOTO_JPG, batch_size=4)

def encode_jpeg_uint8(img_m11):
    img = (img_m11 + 1.0) / 2.0 
    img = tf.clip_by_value(img, 0.0, 1.0)
    img = tf.image.convert_image_dtype(img, tf.uint8)
    return tf.io.encode_jpeg(img, format='rgb', quality=95)

def generate_to_zip(generator, dataset, target_count, zip_path):
    written = 0
    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_STORED) as zf:
        for batch in dataset:
            fakes = generator(batch, training=False)
            for i in range(fakes.shape[0]):
                if written >= target_count: break
                img_bytes = encode_jpeg_uint8(fakes[i]).numpy()
                zf.writestr(f"image_{written:06d}.jpg", img_bytes)
                written += 1
            if written >= target_count: break
        
        if written < target_count:
            print(f"Dogenerowuję {target_count - written} obrazów z lekką augmentacją…")
            for batch in dataset:
                imgs = (batch + 1.0) / 2.0  # [0,1]
                imgs = tf.image.random_flip_left_right(imgs)
                imgs = tf.image.random_brightness(imgs, max_delta=0.05)
                imgs = tf.image.random_contrast(imgs, 0.95, 1.05)
                imgs = tf.clip_by_value(imgs, 0.0, 1.0)
                imgs = imgs * 2.0 - 1.0  # z powrotem [-1,1]
                fakes = generator(imgs, training=False)
                for i in range(fakes.shape[0]):
                    if written >= target_count: break
                    img_bytes = encode_jpeg_uint8(fakes[i]).numpy()
                    zf.writestr(f"image_{written:06d}.jpg", img_bytes)
                    written += 1
                if written >= target_count: break
    return written

TARGET_COUNT = 9500
ZIP_PATH = "/kaggle/working/images.zip"

generated_count = generate_to_zip(G_p2m, photo_ds_infer, TARGET_COUNT, ZIP_PATH)
print(f"✅ Zapisano {generated_count} obrazów do: {ZIP_PATH}")
